<center>
    <img src="../imagens/logo_APL.png" width="300" alt="APL logo"  />
</center>

# Predição de desgaste em uma máquina CNC

**Bem vindo!** Neste material você aplicará todo o conhecimento obtido ao longo de nosso curso a um projeto real em ciência de dados para **predição de desgaste em uma máquina CNC**. Ao final, espera-se que você entenda os principais passos em cada uma das etapas desse processo.

<h2>Conteúdo:</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li> Introdução </li>
    <li> Entendendo e Carregando o Dataset </li> 
    <li> Análise Exploratória dos Dados </li>
    <li> Regressores (Matriz de entrada) e Variável Alvo (vetor de saídas) </li>  
    <li> Dividir o Dataset: Treinamento e Validação </li>   
    <li> Algoritmo de Machine Learning </li>  
    <li> Avaliação </li>  
    <li> Representação: Árvore de Decisão </li>  
</ul>
</div>

<hr>

# Introdução

O projeto escolhido consiste na previsão de desgaste de broca em uma máquina CNC de acordo com várias medições de variáveis elétricas e mecânicas feitas ao longo do processo de fresamento de uma peça. Esse modelo pode ser útil na automação da troca de brocas, tornando o processo mais eficiente e possivelmente independente da intervenção humana. 

Para treinar o modelo usaremos o dataset disponibilizado pela [University of Michigan](https://www.kaggle.com/shasun/tool-wear-detection-in-cnc-mill) em que foram gravados dados de 18 experimentos para fresamento de um tabloide de cera num formato de "S”, de acordo com a figura:


<center>
    <img src="../imagens/S_Shape.png" width="200" alt="S Shape"  />
</center>

Inicialmente, vamos carregar as bibliotecas:

In [ ]:
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Entendendo e Carregando o Dataset

O dataset consiste de 18 experimentos de usinagem em blocos de cera nas dimensões 5.08 x 5.08 x 3.81 cm. 

Os parâmetros variados em cada experimento foram:
- Condição da broca como desgastada ou não
- Velocidade do spindle relativo à peça
- Pressão que segura a peça

As informações de cada experimento podem ser divididas em:
* Entradas: i) No: número do experimentoç ii) `material`: cera (wax); iii) `feed_rate`: velocidade do spindle; e iv) - `clamp_pressure`: pressão que segura a peça.
* Saídas: `tool_condition`: unworn ou wornç ii) `machining_completed`: indicador de se a usinagem foi completada; iii) `passed_visual_inspection`: indicador de se a peça passou por inspeção visual.

Cada arquivo do experimento contém dados de séries temporais relativas a várias medições de variáveis elétricas e mecânicas feitas ao longo do processo de fresamento de uma peça, coletados com um período de amostragem de 100 ms.

Agora vamos carregar os dados do arquivo CSV:

In [ ]:
prefix = 'https://raw.githubusercontent.com/APL-Data-Intelligence/AceleraAI/main/Curso_NEED/datasets/cnc_dataset/'
df=pd.read_csv(prefix + 'train.csv')
df.head()

Vamos usar o método `.info()` para descrição sucinta das principais variáveis de nosso dataframe:

In [ ]:
print(df.info())

# Análise Exploratória dos Dados

Como você pode perceber, algumas das *features* desse dataset (**tool_condition**,  **machining_finalized**, **feedrate** e **clamp_pressure**) são variáveis categóricas. Infelizmente, alguns algoritmos da `Scikit-Learn` não lidam com variáveis categóricas. 

Mas podemos converter essas variáveis categóricas em valores numéricos. Para tanto, vamos usar o método `preprocessing.LabelEncoder()` da biblioteca `Scikit-Learn`.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le2=LabelEncoder()
le2.fit(df['tool_condition'])
df['Encoded_tool_condition']=le2.transform(df['tool_condition'])

le3=LabelEncoder()
le3.fit(df['machining_finalized'])
df['Encoded_machining_finalized']=le3.transform(df['machining_finalized'])

le4=LabelEncoder()
le4.fit(df['feedrate'])
df['Encoded_feedrate']=le4.transform(df['feedrate'])

le5=LabelEncoder()
le5.fit(df['clamp_pressure'])
df['Encoded_clamp_pressure']=le5.transform(df['clamp_pressure'])

Vamos agora utilizar o método `.drop()` para remover algumas *features* que não serão mais necessárias. Oberseve que são aquelas *features* que foram convertidas de variáveis categóricas para variáveis numéricas:

In [ ]:
df.drop(['passed_visual_inspection','tool_condition','material','machining_finalized'],axis=1,inplace=True)

Agora vamos concatenar todos os 18 experimentos de forma a obter um único dataset. Para tanto, vamos percorrer cada um dos 18 experimentos e adicionar as *features* **Encoded_feedrate** e **Encoded_clamp_pressure** (novamente, fazemos a conversão de variáveis categóricas em numéricas).

In [ ]:
frames = []
for i in range(1,19):
    ex_num = '0' + str(i) if i < 10 else str(i)
    frame = pd.read_csv(prefix + "experiment_{}.csv".format(ex_num))

    ex_result_row = df[df['No'] == i]

    le6=LabelEncoder()
    le6.fit(frame['Machining_Process'])
    frame['Encoded_Machining_Process']=le6.transform(frame['Machining_Process'])
    frame.drop('Machining_Process', axis=1, inplace=True)

    frame['Encoded_feedrate'] = ex_result_row.iloc[0]['Encoded_feedrate']
    frame['Encoded_tool_condition'] =ex_result_row.iloc[0]['Encoded_tool_condition']
    frame['Encoded_clamp_pressure'] = ex_result_row.iloc[0]['Encoded_clamp_pressure']
    frames.append(frame)



Agora vamos atribuir a lista `frames` (que corresponde aos 18 experimentos) a um novo dataframe `df_final`:

In [ ]:
df_final = pd.concat(frames, ignore_index=True)
print("Dimensão do Dataset:", df_final.shape)
print("Numero de amostras: ", df_final.shape[0], " (numero de linhas do dataframe).")
print("Numero de features: ", df_final.shape[1], " (numero de colunas do dataframe).")

Vamos, novamente, utilizar o método `.drop()` para remover algumas variáveis que não foram medidas ao longo do experimento:

In [ ]:
df_final.drop(['Z1_CurrentFeedback','Z1_DCBusVoltage','Z1_OutputCurrent',
               'Z1_OutputVoltage','S1_SystemInertia'],axis=1,inplace=True)

In [ ]:
print(">>>>> Heatmap of frames after drop column <<<<<")
plt.figure(figsize=(50,50))
sns.heatmap(df_final.corr(),linewidths=.1,annot=True)
plt.show()

Pela análise da matriz anterior podemos observar que, de forma geral, existe baixa correlação entre as *features*. 

## Separar as Colunas do Dataframe 
#### Regressores (Matriz de entrada) vs Variável Alvo (vetor de saídas).

Inicialmente, vamos definir os regressores (matriz de entrada), `X`:

In [ ]:
X = df_final.drop(['Encoded_tool_condition'], axis=1)
print("Dimensão dos Regressores (matriz de entradas):", X.shape)

In [ ]:
y = df_final.Encoded_tool_condition
print("Dimensão da variável alvo (vetor de saídas):", y.shape)

## Dividir o Dataset: Treinamento e Validação

Vamos dividir nosso dataset em dois subconjuntos: 1) dados de treinamento; e 2) dados de validação.  Para tanto, vamos utilizar o método `train_test_split`.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

## Algoritmo de Machine Learning

Vamos calcular o tempo de execução de cada algoritmo. Para tanto, vamos usar o método `datatime`.

In [ ]:
from datetime import datetime

Vamos seguir os passos para definição dos nossos algoritmos. 

Nesse notebook vamos comparar os três algoritmos de Machine Learning estudados ao longo de nosso curso:
1. Árvore de Decisão;
2. SMV; e
3. KNN

In [ ]:
########  Árvore de Decisão ###################
# inciando o temporizador
time_start=datetime.now()

# Passo 1: Importando o classificador 
from sklearn.tree import DecisionTreeClassifier
# Passo 2: Instanciando o Estimador.
DT_clf = DecisionTreeClassifier()
# Passo 3: Ajustar os parâmetros do modelo a partir dos dados
DT_clf.fit(X_train, y_train)
# Passo 4: Fazer a predição a partir de um novo comjunto de amostras:
DT_y_pred = DT_clf.predict(X_test)
print('==========   Árvore de Decisão   =======')
# finalizando o temporizador
time_end=datetime.now()
e= time_end - time_start
print("Tempo de execução: ",e)

################  SVM   ###################
# inciando o temporizador
time_start=datetime.now()

# Passo 1: Importando o classificador
from sklearn import svm
# Passo 2: Instanciando o Estimador.
SVM_clf = svm.SVC()
# Passo 3: Ajustar os parâmetros do modelo a partir dos dados
SVM_clf.fit(X_train, y_train)
# Passo 4: Fazer a predição a partir de um novo comjunto de amostras:
SVM_y_pred = SVM_clf.predict(X_test)
print('==========   Support Vector Machine   =======')
# finalizando o temporizador
time_end=datetime.now()
e= time_end - time_start
print("Tempo de execução: ",e)


################  KNN   ###################
# inciando o temporizador
time_start=datetime.now()

# Passo 1: Importando o classificador
from sklearn.neighbors import KNeighborsClassifier
# Passo 2: Instanciando o Estimador.
KNN_clf = KNeighborsClassifier()
# Passo 3: Ajustar os parâmetros do modelo a partir dos dados
KNN_clf.fit(X_train, y_train)
# Passo 4: Fazer a predição a partir de um novo comjunto de amostras:
KNN_y_pred = KNN_clf.predict(X_test)
print('==========   K Nearest Neighbours   =======')
# finalizando o temporizador
time_end=datetime.now()
e= time_end - time_start
print("Tempo de execução: ",e)

## Avaliação de Desempenho

Para cálculo da métrica de desempenho vamos usar o método `metrics` da Biblioteca `scikit-learn`.

In [ ]:
from sklearn import metrics

Como vimos no módulo anterior, o método `metrics.accuracy_score` calcula a acurácia do modelo, ou seja, o percentual de valores previstos que foram iguais aos valores reais. Vamos utilizá-lo para comapração de desempenho entre os modelos.

In [ ]:
print('==========   Árvore de Decisão   =======')
print("Precisão no conjunto de treino: ", 100*metrics.accuracy_score(y_train, DT_clf.predict(X_train)), '%.')
print("Precisão no conjunto de teste: ", 100*metrics.accuracy_score(y_test, DT_y_pred), '%.')


print('==========   Support Vector Machine   =======')
print("Precisão no conjunto de treino: ", 100*metrics.accuracy_score(y_train, SVM_clf.predict(X_train)), '%.')
print("Precisão no conjunto de teste: ", 100*metrics.accuracy_score(y_test, SVM_y_pred), '%.')


print('==========   K Nearest Neighbours   =======')
print("Precisão no conjunto de treino: ", 100*metrics.accuracy_score(y_train, KNN_clf.predict(X_train)), '%.')
print("Precisão no conjunto de teste: ", 100*metrics.accuracy_score(y_test, KNN_y_pred), '%.')

## Representação: Árvore de Decisão

In [ ]:
from sklearn import tree
plt.figure(figsize=(30,30))
tree.plot_tree(DT_clf)
plt.show()

<hr>

## Direitos Autorais

[APL Data Intelligence](https://linktr.ee/APLdataintelligence)&#8482;  2021. Este notebook Python e seu código fonte estão liberados sob os termos da [Licença do MIT](https://bigdatauniversity.com/mit-license/).